# Random Forest
Train on 6-class cancer classification.  
Number of trees = 500 (not default 100).  
Input is the nucleus (no RBC) rollup, training set only.  
Run 5-fold cross validation.  
Print the mean accuracy. 

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-13 12:46:03.503800
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/Adjeroh/Naved/CP_80K/'  # Alien
BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K\\'  # Windows
CLASSES=range(0,6)  # use all 6 classes
#CLASSES=[5]  # JUST FOR TESTING
MODELS_DIR='/home/jrm/Adjeroh/Naved/CP_80K/models/RandomForest.31'  # Alien
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.31'  # Windows

In [4]:
# These patches were used to tune the CellProfiler pipeline
EXCLUDE = [ [311], [12], [30,41], [87], [67], [26,46] ]

In [5]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        exclude = list(EXCLUDE[i])
        print('Exclude',exclude)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        Xnuc = Xnuc.drop( Xnuc.index[exclude] )
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        #Xrbc = cp.get_RBC_rollup()
        #Xrbc = Xrbc.drop( Xrbc.index[exclude] )
        #set_prefix('Rbc_')
        #Xrbc.rename(get_prefix,axis='columns',inplace=True)
        #Xi = Xnuc.join(Xrbc,how='outer')
        Xi = Xnuc
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-13 12:46:04.835786
2022-06-13 12:46:04.835786
Process class 0
Exclude [311]
Train: 37 participants, 98 WSI, 37561 patches.
Test: 9 participants, 19 WSI, 7405 patches.
2022-06-13 12:47:08.724227
Process class 1
Exclude [12]
Train: 17 participants, 28 WSI, 10925 patches.
Test: 4 participants, 5 WSI, 1977 patches.
2022-06-13 12:47:27.172600
Process class 2
Exclude [30, 41]
Train: 14 participants, 30 WSI, 11138 patches.
Test: 3 participants, 7 WSI, 2812 patches.
2022-06-13 12:47:47.796531
Process class 3
Exclude [87]
Train: 11 participants, 13 WSI, 5006 patches.
Test: 3 participants, 4 WSI, 1358 patches.
2022-06-13 12:47:57.826203
Process class 4
Exclude [67]
Train: 6 participants, 7 WSI, 2809 patches.
Test: 1 participants, 1 WSI, 388 patches.
2022-06-13 12:48:02.991842
Process class 5
Exclude [26, 46]
Train: 2 participants, 3 WSI, 1195 patches.
Test: 1 participants, 1 WSI, 396 patches.


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Nuc_Texture_Variance_Hematoxylin_7_02_256_75%,Nuc_Texture_Variance_Hematoxylin_7_02_256_max,Nuc_Texture_Variance_Hematoxylin_7_03_256_count,Nuc_Texture_Variance_Hematoxylin_7_03_256_mean,Nuc_Texture_Variance_Hematoxylin_7_03_256_std,Nuc_Texture_Variance_Hematoxylin_7_03_256_min,Nuc_Texture_Variance_Hematoxylin_7_03_256_25%,Nuc_Texture_Variance_Hematoxylin_7_03_256_50%,Nuc_Texture_Variance_Hematoxylin_7_03_256_75%,Nuc_Texture_Variance_Hematoxylin_7_03_256_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
1985,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,268.526316,...,2087.897893,4326.369483,19.0,1683.670886,981.495225,461.972222,1050.266691,1663.409763,1962.973521,4426.740493
1986,15.0,8.0,4.472136,1.0,4.50,8.0,11.50,15.0,15.0,239.533333,...,2850.884277,6383.591712,15.0,1802.715635,1755.544471,246.775813,758.208678,1417.936465,2382.863463,7034.170928
1987,5.0,3.0,1.581139,1.0,2.00,3.0,4.00,5.0,5.0,223.000000,...,1965.130720,2106.346221,5.0,1375.321560,349.776862,830.582562,1290.381836,1439.944707,1553.926236,1761.772461
1988,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,242.250000,...,3445.921264,4521.712604,4.0,2192.904696,1842.641020,430.569878,764.332977,2029.682281,3458.253999,4281.684343
1989,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,250.736842,...,3180.440032,3998.171002,19.0,1968.015393,1221.691042,353.551038,842.353209,1840.896385,2764.530517,4240.190001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,2645.755171,5002.663831,7.0,2072.810441,1442.119639,682.306939,946.916962,2007.931412,2641.212717,4643.175378
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,2697.422264,3144.417339,17.0,2120.125661,784.452285,435.222222,1636.718754,2149.951462,2783.133279,3362.958125
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,2196.200883,2971.276407,19.0,1731.620347,639.473204,677.151060,1348.982274,1626.336416,2185.555700,2992.328549


In [6]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-13 12:48:06.464996
Shuffle...


In [7]:
# Note we tried this but always got "has no attribute oob_score_":
# model = RandomForestClassifier(bootstrap=True,oob_score=True)  # out-of-bag score to estimate generalizability
# <train>
# print('oob_score',model.oob_score_)

In [8]:
print(datetime.datetime.now())
print('Cross-validation...')
model = RandomForestClassifier(n_estimators=500)   # default 100
rf1 = RF_Util()
rf1.set_model(model)
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-13 12:48:07.613659
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 6.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.8min remaining:    0.0s


[CV] END .................................................... total time= 6.8min
[CV] END .................................................... total time= 6.9min
[CV] END .................................................... total time= 6.8min
[CV] END .................................................... total time= 6.9min
[0.81023823 0.81123841 0.8097836  0.82024004 0.81631354]
mean 0.8136 +/- 0.0041


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 34.2min finished


In [9]:
print(datetime.datetime.now())
model = rf1.get_model()
joblib.dump(model,MODELS_DIR)

2022-06-13 13:22:19.597124


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.31']